In [1]:
import numpy as np


In [2]:
import pandas as pd
df=  pd.read_csv('../data/processed/cleaned_dataset.csv')


In [3]:
import os
os.chdir('..')

In [ ]:
from src.preprocessing.feature_engineering import FeatureEngineer

In [6]:
cfg = {
    'cat_encoders' : {
        'S_NAME': r'C:\Users\mew\Projects\autotickets-rag-bot\models\vectorizers\ordinal_encoder_S_NAME.pkl',
        'REG': r'C:\Users\mew\Projects\autotickets-rag-bot\models\vectorizers\ordinal_encoder_REG.pkl',
        'SOURCE': r'C:\Users\mew\Projects\autotickets-rag-bot\models\vectorizers\ordinal_encoder_SOURCE.pkl'
        },
    'embedding_model_path': './models/embeddings/deepvk_USER-bge-m3',
    'sensitive_patterns': {
        "EMAIL": r'\b[A-Za-z0-9._%+-]+@(?:[A-Za-z0-9-]+\.)+[A-Za-z]{2,}\b|<".+?"@[^>]+>',
        "INN": r'(?i)инн\s*\d{5,12}',
        "PHONE": r'\b(?:\+7|8)?[\s-]?\(?\d{3,4}\)?[\s-]?\d{2,3}[\s-]?\d{2}[\s-]?\d{2}(?:\s*\(доб\.\s*\d+\))?\b',
        # "PHONE_EXT": r'\b8\s?\d{3,4}[\s-]?\d{2}[\s-]?\d{2}[\s-]?\d{2}(?:\s*\(доб\.\s*\d+\))?',
        "VIN": r'\b[A-HJ-NPR-Z0-9]{17}\b',
        "INCEDENT": r'\bIM\d{8-12}\b',
        "REG_NUMBER": r'\b[АВЕКМНОРСТУХ]{1,3}\d{3,4}[АВЕКМНОРСТУХ]{2}\b',
        "CASE_NO": r'\b(?:[CcSs]D|T)\d{6,10}\b',
        "APPEAL_NO": r'\b\d/\d{9,12}\b',
        "DOC_NO": r'№\s?\d{1,6}([/\\-]?\d{1,6})?([А-ЯA-Z])?',
        "LONG_ID": r'\b\d{9,}\b|\b[a-f0-9]{32,128}\b|(?:UID|GlndID|GUID)[: ]?[0-9a-fA-F\-]{16,128}',
        "IP": r'\b(?:\d{1,3}\.){3}\d{1,3}\b',
        "DATE": r'\b\d{2}[./-]\d{2}[./-]\d{2,4}(?:\s?г\.?)?',
        "FIO": r'\b[А-ЯЁ][а-яё]+ [А-ЯЁ]\.[А-ЯЁ]\.(?=[\s,.)/]|$)',
        "USERNAME": r'\b[a-zA-Z][a-zA-Z0-9]{3,15}\d\b',
        "TOKEN": r'\b[a-f0-9]{16,64}\b|\b(?=.*[A-Z])(?=.*[a-z])(?=.*\d)[A-Za-z0-9]{12,64}\b',
        "URL": r'https?://[^\s]+',
    },
    'categorical': ['S_NAME', 'REG', 'SOURCE'],
    'time': ['OPEN_TIME_'],
    'text_col': 'QUESTION_clean'
}
fe = FeatureEngineer(cfg)

In [5]:
df[:10]['QUESTION_clean']

0                                      сброс пароля УЗ
1                                            [NO TEXT]
2                                            [NO TEXT]
3               Легалов. Удаление должностной позиции.
4    не работает модуль "Розыск ТС и интерпол", вых...
5    Доброго времени суток. С праздником. Прошу вне...
6    Здравствуйте. Невозможно в резолюции на поступ...
7                                            [NO TEXT]
8    Заявка на предоставление доступа в СО ДЧ сотру...
9    обратилась гр. [FIO], [DATE]р., по вопросу обм...
Name: QUESTION_clean, dtype: object

In [28]:
a = fe.transform(df[:10])

In [ ]:
fe.to_numpy(fe.transform(df[:1]), [''])

Index(['S_NAME', 'QUESTION', 'ANSWER', 'OPEN_TIME_', 'RESOLVE_TIME_',
       'CLOSE_TIME_', 'ATC_NEXT_BREACH_', 'TITLE', 'SUBCATEGORY', 'REG',
       'PRIORITY', 'AVARIYA', 'SOURCE', 'QUESTION_clean', 'ANSWER_clean',
       'SUBCATEGORY_clean', 'S_NAME_enc', 'REG_enc', 'SOURCE_enc',
       'OPEN_TIME__hour_sin', 'OPEN_TIME__hour_cos',
       'OPEN_TIME__day_of_week_sin', 'OPEN_TIME__day_of_week_cos',
       'is_weekend', 'OPEN_TIME__day_of_month_sin',
       'OPEN_TIME__day_of_month_cos', 'QUESTION_clean_emb', 'HAS_EMAIL',
       'HAS_INN', 'HAS_PHONE', 'HAS_VIN', 'HAS_INCEDENT', 'HAS_REG_NUMBER',
       'HAS_CASE_NO', 'HAS_APPEAL_NO', 'HAS_DOC_NO', 'HAS_LONG_ID', 'HAS_IP',
       'HAS_DATE', 'HAS_FIO', 'HAS_USERNAME', 'HAS_TOKEN', 'HAS_URL',
       'HAS_TEXT'],
      dtype='object')

In [2]:
pp = np.load('../data/splits/train.npy')

In [3]:
pp.shape

(535399,)

In [6]:
a = np.load("../data/features/cat_features.npz")

In [12]:
for i in range(a["features"].shape[1]):
    print(len(np.unique(a["features"][:, i])))

22
862
5


In [1]:
import torch_directml

In [31]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))
from src.data_loader import load_features
from src.trainers import train_on_split

import pandas as pd
import numpy as np
from src.model_utils import load_model_bundle
from src.data_loader import load_split, load_features, load_target
from src.config import MODELS_DIR, TARGETS
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from pathlib import Path
import sys
import joblib
from sklearn.metrics import classification_report
from src.config import DEFAULT_FEATURES


In [9]:
tfidfn = np.load("../data/features/question_tfidf.npz", allow_pickle=True)

In [22]:
tfidfn['features'].item().shape

(918476, 8000)

In [33]:
features = DEFAULT_FEATURES
target_col = TARGETS["subcategory"]

train_split = load_split("train")
val_split = load_split("val")

x_cat = load_features(['cat_features.npz'], output_format="dense")[train_split]
x_time = load_features(['time_features.npz'], output_format="dense")[train_split]
x_tfidf = load_features(['question_tfidf.npz'], output_format="dense")[train_split]


IndexError: index 123332 is out of bounds for axis 0 with size 1